In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
primary = pd.read_csv("data/2016_presidential_election/primary_results.csv",sep=",")
counties = pd.read_csv("data/2016_presidential_election/county_facts.csv",sep=",")

In [ ]:
print(primary.shape)	#라인수, 컬럼수 확인
print(primary.columns) 	#컬럼명 확인
primary.head()	#상위 5개 행의 데이터 확인

In [ ]:
primary['party'].unique()

In [ ]:
print(counties.shape)
print(counties.columns)
counties.head()

### 각 후보별 전체 득표수 계산하기

In [ ]:
candidate_to_votes_s = primary.groupby("candidate")["votes"].sum().sort_values()

In [ ]:
candidate_to_votes_s

In [ ]:
plt.figure(figsize=(20,10))
candidate_to_votes_s.plot(kind="barh", fontsize=8)

### 각 주별, 각 정당의 득표 비율 계산하기
### 각 주별, 정당별 득표수 계산하기

In [ ]:
state_party_to_votes_s = primary.groupby(["state", "party"])["votes"].sum()
state_party_to_votes_s

In [ ]:
#각 주별 득표수
state_to_votes_s = primary.groupby("state")["votes"].sum()
state_to_votes_s

### state별, party별 득표수의 합의 퍼센티지 구함

In [ ]:
state_party_to_vote_pcts_s = state_party_to_votes_s / state_to_votes_s
state_party_to_vote_pcts_s

In [ ]:
print(type(state_party_to_vote_pcts_s))
print(state_party_to_vote_pcts_s.index)

In [ ]:
#인덱스로 되어 있던 party(정당) : 인덱스를 칼럼으로 보낸것임 - 최하위 인덱스를 칼럼으로 보냄
state_party_to_vote_pcts_df = state_party_to_vote_pcts_s.unstack()
state_party_to_vote_pcts_df

In [ ]:
print(type(state_party_to_vote_pcts_df))
print(state_party_to_vote_pcts_df.index)
print(state_party_to_vote_pcts_df.columns)

In [ ]:
figure, axes1 = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18,12)
#plt.figure(figsize=(100,40))
state_party_to_vote_pcts_df.plot(kind="barh", stacked=True, fontsize=10, ax=axes1)

## primary와 counties 를 merge하는 연습하기

In [ ]:
#merge 연습 fips 칼럼 기준으로 merge
pd.merge(primary, counties, on="fips")

In [ ]:
#primary의 모든 컬럼과 counties의 'fips'와 'RHI425214' 컬럼  join
pd.merge(primary, counties[['fips','RHI425214']], on="fips")

In [ ]:
#primary의 'fips','party' 컬럼과 counties의 'fips'와 'RHI425214' 컬럼  outer join
#pd.merge(primary[['fips','party']], counties[['fips','RHI425214']], on="fips") #13211
pd.merge(primary[['fips','party']], counties[['fips','RHI425214']], on="fips",how='outer') #14358

In [ ]:
#100개의 row 만 가져와서 merge 함
pd.merge(primary.loc[:100,['state','fips','county','party']], counties.loc[:100,['fips','area_name','RHI425214']], on="fips",how='left')

In [ ]:
primary['state'].unique()

In [ ]:
primary['state'].value_counts()

In [ ]:
primary.loc[primary['state'] == 'Maine',:]

In [ ]:
#Maine state 인 row 의 merge
pd.merge(primary.loc[primary['state'] == 'Maine',['state','fips','county','party','candidate']],\
         counties.loc[:,['fips','area_name','RHI425214']], on="fips",how='left')

In [ ]:
#fips 컬럼을 index로 변경함
counties2 = counties.set_index("fips")
counties2.head()

In [ ]:
#인덱스 기준 merging
pd.merge(primary, counties2[['area_name','PST045214']], left_on="fips", right_index=True).head()
#pd.merge(primary[['fips','county','party']], counties2[['area_name','PST045214']], left_on="fips", right_index=True)

In [ ]:
primary.head()

In [ ]:
#계층적 인덱싱 만들기
primary2 = primary.set_index(["state","county"])

In [ ]:
primary2.head()

In [ ]:
#3개의 컬럼을 인덱스로 변경해줌
primary2 = primary.set_index(["state","county","party"])

In [ ]:
primary2.head()

In [ ]:
#기존 열을 그대로 유지하고 싶다면 drop=False
primary2 = primary.set_index(["state","county","party"],drop=False)

In [ ]:
#각 후보가 당선된 county의 평균 백인 유권자 비율 조사하기 하기전에 연습할 내용

In [ ]:
#fips 마다 후보자들의 득표수가 있다는 것을 확인함
primary.loc[primary["fips"] == 1005,:].sort_values("votes",ascending=False)

In [ ]:
primary.loc[primary["fips"] == 1005,:].sort_values("votes",ascending=False).iloc[0]

In [ ]:
#fips별 최대 득표수
primary.groupby("fips")["votes"].max()

In [ ]:
#각 후보가 당선된 county의 평균 백인 유권자 비율 조사하기
func = lambda agg_df: agg_df.sort_values("votes", ascending=False).iloc[0]
winners = primary.groupby("fips").agg(func)

In [ ]:
winners.head()

In [ ]:
#primary와 counties 데이터프레임을 merge 한다.  White alone(백인)
winners_county_races = pd.merge(winners, counties[["fips", "RHI825214"]], left_index=True, right_on="fips", how="left")

In [ ]:
winners_county_races.head()

In [ ]:
#컬럼명  변경
winners_county_races = winners_county_races.rename(columns={"RHI825214":"white_pcts"})

In [ ]:
winners_county_races.head()

In [ ]:
winners_county_white_pcts = winners_county_races.groupby(["party", "candidate"])["white_pcts"].mean()

In [ ]:
winners_county_white_pcts

In [ ]:
#plt.figure()
plt.figure(figsize=(24,10))
ax = winners_county_white_pcts.plot(kind="barh", fontsize=10)
ax.set_xlim([50, 100])
plt.tight_layout()

In [ ]:
#이와 같은 groupby 구문 대신에 위와 같은 pivot_table() 함수를 사용할 수 있다. 
#state별 candidate의 득표수의 합
primary.groupby(["state","candidate"])["votes"].sum()

In [ ]:
total_votes = primary.pivot_table(values="votes", index="state", 
                    columns="candidate", aggfunc="sum",fill_value=0)

In [ ]:
total_votes

In [ ]:
#state_abbreviation별, party 별 투표비율의 평균
primary.groupby(['state_abbreviation','party'])["fraction_votes"].mean()

In [ ]:
#pivot_table 사용
mean_frac_votes=primary.pivot_table(values="fraction_votes", index="state_abbreviation", 
                    columns="party", aggfunc="mean")

In [ ]:
mean_frac_votes